In [1]:
import pandas as pd 
import process
import numpy as np 
# Jerome path : r'C:\Users\33640\OneDrive\Documents\GitHub\Portfolio_clustering_project\Data\DataBase.csv'
# Nail path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DataBase.csv'
df = pd.read_csv(r'/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DataBase.csv')

df.set_index('ticker', inplace=True)

df.columns = pd.to_datetime(df.columns.str[1:], format='%Y%m%d').strftime('%d/%m/%Y')

df_cleaned = df.fillna(0) # Utilisez la méthode fillna(0) pour remplacer les NaN par 0

df_cleaned = df_cleaned.transpose() ## WE WANT COLUMNS TO BE VECTOR OF RETURN FOR A GIVEN TICKER

We denote by $\bm{X}$ the $d \times n$ matrix of observations \textit{i.e.}

$$
\bm{X} = 
\begin{bmatrix}
    r_{1}^{(1)} & r_{1}^{(2)} & \ldots & r_{1}^{(n)} \\
    r_{2}^{(1)} & r_{2}^{(2)} & \ldots & r_{2}^{(n)} \\
    \vdots & \vdots & \ddots & \vdots \\
    r_{d}^{(1)} & r_{d}^{(2)} & \ldots & r_{d}^{(n)} \\
\end{bmatrix} 
= \begin{bmatrix} \mathbf{r}^{(1)} | ... |\mathbf{r}^{(n)}\end{bmatrix}
\in \mathbb{R}^{d\times n}
$$

where:
- $d$ corresponds to the number of days
- $n$ corresponds to the number of stocks

In general, a standard sample covariance can be generalized to include some arbitrary weight profile assigned along the time dimension. In particular, it is expressed in the following form

\begin{equation}
\bm{S_W} := \frac{1}{d} \bm{X}'W\bm{X} \in \mathbb{R}^{n\times n}
\end{equation}\\

The EWA-SC as defined can be written as a weighted sample covariance matrix if we define the matrix of weighted $W$ to be: 

\begin{align*}
    W_{t,k} =
    \begin{cases}
        d\frac{1 - \beta}{1-\beta^d} \beta^{d-t} & \text{if } t = k \\
        0 & \text{otherwise}
    \end{cases}
\end{align*}

If we define the auxiliary observation matrix: 

\begin{align}
    \tilde{\bm{{X}}} := \bm{W}^\frac{1}{2} \bm{X}
\end{align}

then we can see that the EWA-SC can be expressed in a similar form as the standard uniformly weighted sample covariance. The advantage of recasting the EWA-SC in this way is that many of the refinements for the standard sample covariance that have been developed over the years are at our disposal; including shrinkage.

In [2]:
######################### 1. We start by randomizing the auxiliary observation matrix  ̃X from Equation (5) along the time axis #########################
def auxilary_matrix(days, beta, df_cleaned):

    ## 1. We extract the data corresponding to the returns of our assets (columns) during these d days (lines)
    X = df_cleaned.iloc[0:days,:] ## shape days * number of stocks

    ## 2. We slightly adjust the matrix of observations to get the auxiliary matrix that puts more weight on recent dates

    W = np.sqrt(np.diag(days * (1 - beta) * beta**(np.arange(days)[::-1]) / (1 - beta**days)))  # Compute the weight matrix
    X_tilde = pd.DataFrame(index=X.index, columns=X.columns, data=np.dot(W, X)).transpose()

    ## 3. We randomize the auxiliary matrix of observations according to the time axis
    # Randomized_X = X_tilde.transpose().sample(frac=1, axis=1, random_state=42) ## we transpose X as we want to have daily observations of the whole dataset !

    return X_tilde

# ---------------------------------------------------------------- TESTS ----------------------------------------------------------------

days = 250
beta = 0.999
X_tilde = auxilary_matrix(days=days, beta=beta, df_cleaned=df_cleaned)
X_tilde

,03/01/2000,04/01/2000,05/01/2000,06/01/2000,07/01/2000,10/01/2000,11/01/2000,12/01/2000,13/01/2000,14/01/2000,...,13/12/2000,14/12/2000,15/12/2000,18/12/2000,19/12/2000,20/12/2000,21/12/2000,22/12/2000,26/12/2000,27/12/2000
ticker,,,,,,,,,,,,,,,,,,,,,
AA,-0.012239,0.009429,0.044739,-0.011008,-0.015155,-0.030173,0.021279,-0.004943,-0.017157,-0.018955,...,0.037058,-0.002082,-0.008524,0.018609,0.040317,-0.051453,0.012609,0.072985,-0.007656,-0.009671
ABM,-0.008622,0.011591,-0.005816,0.000000,0.002906,0.000000,-0.008755,0.002947,-0.026964,0.011710,...,0.017241,-0.004303,0.002164,0.015092,0.008568,-0.006436,0.000000,0.052943,-0.029570,0.004304
ABT,-0.006679,-0.012004,0.010437,0.030593,0.026866,-0.019806,0.010212,-0.020509,-0.008684,0.000000,...,0.023227,-0.048372,0.030120,0.035326,0.026674,-0.025497,-0.013694,-0.018056,0.026508,-0.005458
ADI,-0.033849,-0.041555,0.013614,-0.026050,0.031644,0.045277,-0.030045,0.032663,-0.019261,0.053811,...,-0.100518,-0.026081,0.020950,-0.057097,0.036648,-0.003907,-0.057160,0.012197,0.012939,0.070431
ADM,0.000000,0.004954,-0.014950,0.010051,0.004936,-0.004913,-0.014900,0.019722,0.000000,0.028712,...,0.000000,0.010586,-0.010234,0.030135,0.004875,-0.009867,0.043721,-0.004794,0.014324,0.018730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XLY,-0.026868,-0.014942,-0.015635,0.004538,0.033271,-0.003860,-0.008233,-0.007799,0.000000,-0.003414,...,-0.020053,0.000678,-0.008340,0.013062,-0.028330,0.021488,0.033561,0.013630,-0.010994,0.043059
XOM,-0.015621,-0.006850,0.035450,0.049661,-0.011006,-0.004901,0.002806,0.002824,0.019541,-0.019494,...,0.011374,-0.015365,-0.004711,0.026397,0.012999,-0.025376,-0.003127,0.023966,0.021202,-0.012793
XRX,0.032064,-0.044716,0.020201,0.007449,0.022010,-0.038799,-0.014900,0.007564,0.022286,-0.024678,...,0.035471,0.087359,0.032261,0.020985,0.051229,-0.043302,-0.117316,0.053774,-0.026896,0.040624


We split the (randomized) auxiliary observations into $K$ non-overlapping folds **of equal size** represented as $\{\mathcal{I}_k | \mathcal{I}_k \subset \{1, ..., K\}\}_{k=1}^K$. Each set indexed by $\mathcal{I}_k$ works as a **"test" fold**, while the remaining observations' indices constitute a **"training" fold**,

In [4]:
from sklearn.model_selection import ShuffleSplit

######################### 2. We then split the (randomized) auxiliary observations into K non-overlapping folds of equal size #########################
def shuffle_split(data, K):
    # Initialize ShuffleSplit
    shuffle_split = ShuffleSplit(n_splits=K, test_size=0.2, random_state=42) 
    # test_size=0.2 : 20% des données pour l'ensemble de test, 80% pour l'ensemble d'entraînement.

    # Create empty list to store splits
    splits = []

    # Perform shuffling and splitting
    for train_index, test_index in shuffle_split.split(data.columns):
        train_fold = [data.columns[i] for i in train_index]
        test_fold = [data.columns[i] for i in test_index]
        splits.append((train_fold, test_fold)) ## attention à cette structure

    return splits

######################### 3. For each K fold configuration, we estimate the sample eigenvectors from the training set #########################
def eigen_sample(data, train_fold): ## we train the data on this test fold

    train_data = data.loc[:, train_fold]

    # Calculer la moyenne de l'ensemble d'entraînement
    mean_train = np.mean(train_data, axis=1)

    # Centrer les données d'entraînement
    centered_train_data = train_data.sub(mean_train, axis=0)

    # Calculer la matrice de covariance des données d'entraînement
    cov_matrix_train = np.cov(centered_train_data) ## size number of assets * number of assets

    # Calculer les vecteurs et valeurs propres de la matrice de covariance
    _, eigenvectors_train = np.linalg.eigh(cov_matrix_train) ## .eigh and not .eig so that the eigenvalues are real 

    return eigenvectors_train


# ---------------------------------------------------------------- TESTS ----------------------------------------------------------------
days = 250
beta = 0.999
X_tilde = auxilary_matrix(days=days, beta=beta, df_cleaned=df_cleaned)
splits = shuffle_split(data=X_tilde, K=20)
eigenvector = eigen_sample(data=X_tilde, train_fold=splits[0][0])
eigenvector

array([[ 0.  ,  0.  ,  0.  , ..., -0.  , -0.05,  0.03],
       [ 0.09,  0.17, -0.18, ..., -0.01,  0.  ,  0.02],
       [ 0.18,  0.02, -0.1 , ..., -0.  , -0.04,  0.01],
       ...,
       [ 0.01,  0.01,  0.02, ...,  0.04,  0.03,  0.03],
       [ 0.01,  0.02, -0.01, ..., -0.02, -0.05,  0.04],
       [-0.  ,  0.01,  0.03, ...,  0.01,  0.  ,  0.01]])

We consider a fixed exponential decay rate $\beta \in (0, 1)$ an its associated EWA-SC $\bm{E}$. Remember we denoted $\bm{\Sigma}$ the "true" and unobserved covariance matrix. Both these matrices are symmetric and thus admit the following spectral decomposition: 

\begin{equation}
\bm{E} = \sum_{i=1}^{n} \hat{\lambda}_i \hat{u}_i \hat{u}_i', \quad \text{and} \quad \bm{\Sigma} = \sum_{i=1}^{n} \lambda_i u_i u_i',
\end{equation}

where $(\hat{\lambda}_1, \ldots, \hat{\lambda}_n; \hat{u}_1, \ldots, \hat{u}_n)$  denotes a system of sample eigenvalues and eigenvectors of $\bm{E}$, and $(\lambda_1, \ldots, \lambda_n; u_1, \ldots, u_n)$ denotes a system of eigenvalues and eigenvectors of the "true" covariance $\bm{\Sigma}$. The eigenvalues are assumed to be sorted in ascending order.

To correct the bias previously mentioned, we consider a specific framework where the sample eigenvalues should be corrected while retaining the sample eigenvectors of the original matrix. This is mathematically tantamount to write:

\begin{equation}
\hat{\bm{\Sigma}} = \sum_{i=1}^{n} \xi_i \hat{u}_i \hat{u}_i',
\end{equation}

where $\bm{\xi} = (\xi_i)_{i=1,...,n}$  is an $n$-dimensional vector that we have to obtain. This framework is somewhat reasonable as, in absence of any **a priori** knowledge about the structure of the covariance matrix, the most natural guess that we have about the population eigenvectors is the sample eigenvectors that we observe. 

In [5]:
######################### 3. For each K fold configuration, we estimate the sample eigenvectors from the training set #########################
def eigen_sample(data, train_fold): ## we train the data on this test fold

    train_data = data.loc[:, train_fold]

    # Calculer la moyenne de l'ensemble d'entraînement
    mean_train = np.mean(train_data, axis=1)

    # Centrer les données d'entraînement
    centered_train_data = train_data.sub(mean_train, axis=0)

    # Calculer la matrice de covariance des données d'entraînement
    cov_matrix_train = np.cov(centered_train_data) ## size number of assets * number of assets

    # Calculer les vecteurs et valeurs propres de la matrice de covariance
    _, eigenvectors_train = np.linalg.eigh(cov_matrix_train) ## .eigh and not .eig so that the eigenvalues are real 

    return eigenvectors_train


- For each $K$ fold configuration, we estimate the sample eigenvectors from the training set and then estimate an N-dimensional vector of out-of-sample variances using the test set and the sample eigenvectors. 

- Finally, we average the out-of-sample variance estimates over $K$ to give us the bias-corrected eigenvalue of the ith sample eigenvector portfolio denoted as $\xi^{\dagger}_i$ for all $i$.

These two last steps are equivalent to introducing the $K$-fold cross-validation estimator:

$$
\xi^{\dagger}_i := \frac{1}{K} \sum_{k=1}^K \sum_{t \in \mathcal{I}_k}  \frac{1}{\lvert \mathcal{I}_k \rvert} \left(\hat{u}_i[k]'\tilde{x}_t \right)^2, \quad \text{for } i = 1, \ldots, n,
$$

where: 
- $\lvert \mathcal{I}_k \rvert$ denotes the cardinality of the kth test set such that each of them is approximately equal in size, that is, $K \lvert \mathcal{I}_k \rvert \approx d$
- Here, $\hat{u}_i[k]$ is the $i$-th sample eigenvector of a sample covariance matrix that is obtained from the training fold, and $\tilde{x}$ is a sample vector of the auxiliary observation matrix from the test fold.

In [6]:
def intra_fold_loss(data, test_fold, sample_eigenvector_i, beta): ## we test the data on this test fold

    ## 1. get the fold cardinality 
    fold_cardinality = len(test_fold)

    ## 2. sample vector of the auxiliary observation matrix from the test fold (inspired from the code above)

    days = len(test_fold)
    X = data.loc[:,test_fold].transpose()

    ## 2. We slightly adjust the matrix of observations to get the auxiliary matrix that puts more weight on recent dates

    W = np.sqrt(np.diag(days * (1 - beta) * beta**(np.arange(days)[::-1]) / (1 - beta**days)))  # Compute the weight matrix
    X_tilde = pd.DataFrame(index=X.index, columns=X.columns, data=np.dot(W, X)).transpose()

    res = (np.dot(sample_eigenvector_i, X_tilde) ** 2) / fold_cardinality
    result = np.sum(res)

    return result

# ---------------------------------------------------------------- TESTS ----------------------------------------------------------------
beta = 0.99
data = X_tilde
sample_eigenvector_i = eigenvector[0]
X = X_tilde.loc[:, splits[0][1]]
test_fold = splits[0][1]
intra_loss = intra_fold_loss(data=data, test_fold=splits[0][1], sample_eigenvector_i=sample_eigenvector_i, beta=beta)
intra_loss

0.0005555442500824901

In [7]:
def average_loss_i(data, splits, index, beta):

    res = 0 ## to stock the overall loss

    for (train_fold, test_fold) in splits:

        ## sur chaque fold, on calcule les sample eigenvectors à partir du training fold correspondant

        sample_eigenvector_i = eigen_sample(data=data, train_fold=train_fold)[index] ## on ne garde que l'eigenvector correspondant au bon index

        ## sur chaque fold, on calcule la perte au sein du fold à partir de l'échantillon de test

        res = res + intra_fold_loss(data=data, test_fold=test_fold, sample_eigenvector_i=sample_eigenvector_i, beta=beta)

    res = res / len(splits) ## we average by the number of folds (which corresponds to the lengths of the splits)

    return res

# ---------------------------------------------------------------- TEST ----------------------------------------------------------------
average_loss_i = average_loss_i(df_cleaned==df_cleaned, days=250, splits=splits, index=0, beta=0.99)

TypeError: average_loss_i() got an unexpected keyword argument 'days'

In [8]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [9]:
from tqdm import tqdm

def eigenvalue_estimator(df_cleaned, days, K, beta):

    data = auxilary_matrix(days=days, beta=beta, df_cleaned=df_cleaned)
    
    splits = shuffle_split(data=data, K=K)

    number_of_stocks = data.shape[0]

    x = np.zeros(number_of_stocks)  # initialisation de x
    for i in tqdm(range(number_of_stocks), desc='Calcul en cours', unit='itération'):
        x[i] = average_loss_i(data=data, splits=splits, index=i, beta=beta)                  
    return x

# ---------------------------------------------------------------- TEST ----------------------------------------------------------------
eigenvalue_estimator = eigenvalue_estimator(df_cleaned=df_cleaned, days=250, K=10, beta=0.99)

Calcul en cours:   0%|          | 0/695 [00:00<?, ?itération/s]

Calcul en cours: 100%|██████████| 695/695 [14:56<00:00,  1.29s/itération]


In [11]:
np.set_printoptions(precision=10)

eigenvalue_estimator

array([0.0005793698, 0.0006946351, 0.0005185658, 0.0005557637,
       0.0007370129, 0.0006402375, 0.0007496107, 0.0005882843,
       0.0006812528, 0.000712892 , 0.0007119005, 0.0006731959,
       0.0006026536, 0.0006816077, 0.0008599714, 0.000699467 ,
       0.0006363773, 0.0006076935, 0.000674776 , 0.0006844753,
       0.0006404914, 0.000710006 , 0.0007268464, 0.00070062  ,
       0.0008231383, 0.0006831348, 0.0006576698, 0.0006986462,
       0.0008627604, 0.0007062771, 0.0007055713, 0.0008411223,
       0.0007038183, 0.0006542016, 0.0007606853, 0.0005879162,
       0.00050023  , 0.0006409119, 0.000617538 , 0.0006124384,
       0.0006893665, 0.000692577 , 0.0007031964, 0.0006046827,
       0.0007651305, 0.0007379627, 0.0007058259, 0.0006575152,
       0.0006693998, 0.000893408 , 0.0005392596, 0.0006265962,
       0.0005682921, 0.0007374273, 0.0007622489, 0.000735299 ,
       0.0005628294, 0.0006271214, 0.0006132104, 0.0006548223,
       0.0006209698, 0.0007186939, 0.0006543316, 0.0006

In [ ]:
def EWA_CV(eigenvalue_estimator, data, days):
    X = df_cleaned.iloc[0:days,:]

In [24]:
days = 250
beta = 0.99

## compute the sample exponential moving average correlation matrix
X = df_cleaned.iloc[0:days,:]
W = np.sqrt(np.diag(days * (1 - beta) * beta**(np.arange(days)[::-1]) / (1 - beta**days)))  # Compute the weight matrix
res1 = np.dot(X.T, W)  # Produit matriciel de X' et W
S = np.dot(res1, X)

## compute the eigenvectors of S

_, eigenvectors = np.linalg.eig(S)

## computes the estimator 

# Tailles des matrices
num_eigenvalues = eigenvalue_estimator.shape[0]
num_features = eigenvectors.shape[0]

# Initialisation de Sigma avec des zéros
Sigma = np.zeros((num_features, num_features), dtype=np.complex128)

# Parcourir chaque vecteur propre et valeur propre
for i in range(num_eigenvalues):
    xi_dagger = eigenvalue_estimator[i]  # Conjugue de xi
    ui = eigenvectors[:, i]  # i-ème vecteur propre

    # Calcul du produit externe xi^† * ui * ui^† et addition à Sigma
    Sigma += np.outer(xi_dagger, ui) @ np.conj(ui).reshape(-1, 1)

# Sigma est maintenant la somme des produits xi^† * ui * ui^†
Sigma = pd.DataFrame(index=df_cleaned.columns, columns=df_cleaned.columns, data=np.real(Sigma))

In [35]:
eigenvectors

array([[-0.028117201 +0.j, -0.067523087 +0.j, -0.0016752846+0.j, ...,
         0.          +0.j,  0.          +0.j,  0.          +0.j],
       [-0.0238146008+0.j, -0.0035941907+0.j,  0.0205087932+0.j, ...,
         0.          +0.j,  0.          +0.j,  0.          +0.j],
       [ 0.0022368459+0.j, -0.044970012 +0.j,  0.0078269369+0.j, ...,
         0.          +0.j,  0.          +0.j,  0.          +0.j],
       ...,
       [-0.0328243624+0.j,  0.0523596618+0.j, -0.051738209 +0.j, ...,
         0.          +0.j,  0.          +0.j,  0.          +0.j],
       [-0.0383655607+0.j, -0.0453393156+0.j,  0.0258260051+0.j, ...,
         0.          +0.j,  0.          +0.j,  0.          +0.j],
       [-0.007976776 +0.j,  0.0027771308+0.j, -0.0078436884+0.j, ...,
         0.          +0.j,  0.          +0.j,  0.          +0.j]])